In [38]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sns
pd.set_option("display.max_columns", 100)
import os
import geojson
import csv
from geojson import Feature, FeatureCollection

### Read data

In [4]:
dfl = pd.read_csv("/Users/nesara/Documents/etc/airbnb/data/all_listings.csv")
dfl.head(2)

/Users/nesara/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,availability_365,calculated_host_listings_count,city,host_id,host_name,id,last_review,latitude,longitude,minimum_nights,name,neighbourhood,neighbourhood_group,number_of_reviews,price,reviews_per_month,room_type
0,0,78,30,Barcelona,71615,Mireia And Maria,18653,NaN,41.409215,2.173868,2,Modern apartment well connected,la Sagrada Família,Eixample,0,151,NaN,Entire home/apt
1,1,70,30,Barcelona,71615,Mireia And Maria,18666,2015-10-10,41.408886,2.185545,3,Flat with Sunny Terrace,el Camp de l'Arpa del Clot,Sant Martí,1,130,0.04,Entire home/apt


In [7]:
excluded_cities = ["Santa Cruz County", "Victoria", "Northern Rivers", "Tasmania" ,"Hong Kong"]

### Currency conversion factors

In [22]:
dfl.loc[dfl["city"]=="New York City", "localcurrency_usd_factor"] = 1
dfl.loc[dfl["city"]=="London", "localcurrency_usd_factor"] = 1.32
dfl.loc[dfl["city"]=="Paris", "localcurrency_usd_factor"] = 1.15
dfl.loc[dfl["city"]=="Sydney", "localcurrency_usd_factor"] = 0.74
dfl.loc[dfl["city"]=="Los Angeles", "localcurrency_usd_factor"] = 1
dfl.loc[dfl["city"]=="Rome", "localcurrency_usd_factor"] = 1.15
dfl.loc[dfl["city"]=="Berlin", "localcurrency_usd_factor"] = 1.15
dfl.loc[dfl["city"]=="Copenhagen", "localcurrency_usd_factor"] = 0.15
dfl.loc[dfl["city"]=="Amsterdam", "localcurrency_usd_factor"] =  1.15
dfl.loc[dfl["city"]=="Barcelona", "localcurrency_usd_factor"] = 1.15
dfl.loc[dfl["city"]=="Madrid", "localcurrency_usd_factor"] = 1.15
dfl.loc[dfl["city"]=="Mallorca", "localcurrency_usd_factor"] = 1.15
dfl.loc[dfl["city"]=="Melbourne", "localcurrency_usd_factor"] = 0.74
dfl.loc[dfl["city"]=="Toronto", "localcurrency_usd_factor"] = 0.75
dfl.loc[dfl["city"]=="Montreal", "localcurrency_usd_factor"] = 0.75
dfl.loc[dfl["city"]=="Austin", "localcurrency_usd_factor"] = 1
dfl.loc[dfl["city"]=="Vienna", "localcurrency_usd_factor"] = 1.15
dfl.loc[dfl["city"]=="Washington, D.C.", "localcurrency_usd_factor"] = 1
dfl.loc[dfl["city"]=="Dublin", "localcurrency_usd_factor"] = 1.32
dfl.loc[dfl["city"]=="Vancouver", "localcurrency_usd_factor"] = 0.75
dfl.loc[dfl["city"]=="San Diego", "localcurrency_usd_factor"] = 1
dfl.loc[dfl["city"]=="Hong Kong", "localcurrency_usd_factor"] = 0.13
dfl.loc[dfl["city"]=="Edinburgh", "localcurrency_usd_factor"] = 1.32
dfl.loc[dfl["city"]=="Brussels", "localcurrency_usd_factor"] = 1.15
dfl.loc[dfl["city"]=="Venice", "localcurrency_usd_factor"] = 1.15
dfl.loc[dfl["city"]=="Nashville", "localcurrency_usd_factor"] = 1
dfl.loc[dfl["city"]=="New Orleans", "localcurrency_usd_factor"] = 1
dfl.loc[dfl["city"]=="Chicago", "localcurrency_usd_factor"] = 1
dfl.loc[dfl["city"]=="Athens", "localcurrency_usd_factor"] = 1.15
dfl.loc[dfl["city"]=="Boston", "localcurrency_usd_factor"] = 1
dfl.loc[dfl["city"]=="San Francisco", "localcurrency_usd_factor"] = 1
dfl.loc[dfl["city"]=="Portland", "localcurrency_usd_factor"] = 1
dfl.loc[dfl["city"]=="Tasmania", "localcurrency_usd_factor"] = 0.74
dfl.loc[dfl["city"]=="Denver", "localcurrency_usd_factor"] = 1
dfl.loc[dfl["city"]=="Seattle", "localcurrency_usd_factor"] = 1
dfl.loc[dfl["city"]=="Geneva", "localcurrency_usd_factor"] = 1
dfl.loc[dfl["city"]=="Northern Rivers", "localcurrency_usd_factor"] = 0.74
dfl.loc[dfl["city"]=="Quebec City", "localcurrency_usd_factor"] = 0.75
dfl.loc[dfl["city"]=="Oakland", "localcurrency_usd_factor"] = 1
dfl.loc[dfl["city"]=="Victoria", "localcurrency_usd_factor"] = 0.74
dfl.loc[dfl["city"]=="Antwerp", "localcurrency_usd_factor"] = 1.15
dfl.loc[dfl["city"]=="Manchester", "localcurrency_usd_factor"] = 1.32
dfl.loc[dfl["city"]=="Asheville", "localcurrency_usd_factor"] = 1
dfl.loc[dfl["city"]=="Santa Cruz County", "localcurrency_usd_factor"] = 1

In [25]:
dfl.loc[dfl["city"]=="New York City", "avg_hotel_price"] = 245
dfl.loc[dfl["city"]=="London", "avg_hotel_price"] = 252
dfl.loc[dfl["city"]=="Paris", "avg_hotel_price"] = 214
dfl.loc[dfl["city"]=="Sydney", "avg_hotel_price"] = 185
dfl.loc[dfl["city"]=="Los Angeles", "avg_hotel_price"] = 175
dfl.loc[dfl["city"]=="Rome", "avg_hotel_price"] = 167
dfl.loc[dfl["city"]=="Berlin", "avg_hotel_price"] = 128
dfl.loc[dfl["city"]=="Copenhagen", "avg_hotel_price"] = 190
dfl.loc[dfl["city"]=="Amsterdam", "avg_hotel_price"] = 191
dfl.loc[dfl["city"]=="Barcelona", "avg_hotel_price"] = 172
dfl.loc[dfl["city"]=="Madrid", "avg_hotel_price"] = 132
#dfl.loc[dfl["city"]=="Mallorca", "avg_hotel_price"] = 
dfl.loc[dfl["city"]=="Melbourne", "avg_hotel_price"] = 177
dfl.loc[dfl["city"]=="Toronto", "avg_hotel_price"] = 148
#dfl.loc[dfl["city"]=="Montreal", "avg_hotel_price"] = 
dfl.loc[dfl["city"]=="Austin", "avg_hotel_price"] = 160
dfl.loc[dfl["city"]=="Vienna", "avg_hotel_price"] = 147
dfl.loc[dfl["city"]=="Washington, D.C.", "avg_hotel_price"] = 173
dfl.loc[dfl["city"]=="Dublin", "avg_hotel_price"] = 140
dfl.loc[dfl["city"]=="Vancouver", "avg_hotel_price"] = 177
dfl.loc[dfl["city"]=="San Diego", "avg_hotel_price"] = 152
#dfl.loc[dfl["city"]=="Hong Kong", "avg_hotel_price"] = 
dfl.loc[dfl["city"]=="Edinburgh", "avg_hotel_price"] = 118
dfl.loc[dfl["city"]=="Brussels", "avg_hotel_price"] = 121
dfl.loc[dfl["city"]=="Venice", "avg_hotel_price"] = 233
dfl.loc[dfl["city"]=="Nashville", "avg_hotel_price"] = 178
dfl.loc[dfl["city"]=="New Orleans", "avg_hotel_price"] = 191
dfl.loc[dfl["city"]=="Chicago", "avg_hotel_price"] = 179
#dfl.loc[dfl["city"]=="Athens", "avg_hotel_price"] = 
dfl.loc[dfl["city"]=="Boston", "avg_hotel_price"] = 213
dfl.loc[dfl["city"]=="San Francisco", "avg_hotel_price"] = 217
dfl.loc[dfl["city"]=="Portland", "avg_hotel_price"] = 149
#dfl.loc[dfl["city"]=="Tasmania", "avg_hotel_price"] = 
#dfl.loc[dfl["city"]=="Denver", "avg_hotel_price"] = 
dfl.loc[dfl["city"]=="Seattle", "avg_hotel_price"] = 195
dfl.loc[dfl["city"]=="Geneva", "avg_hotel_price"] = 243
#dfl.loc[dfl["city"]=="Northern Rivers", "avg_hotel_price"] = 
#dfl.loc[dfl["city"]=="Quebec City", "avg_hotel_price"] = 
#dfl.loc[dfl["city"]=="Oakland", "avg_hotel_price"] = 
#dfl.loc[dfl["city"]=="Victoria", "avg_hotel_price"] = 
#dfl.loc[dfl["city"]=="Antwerp", "avg_hotel_price"] = 
#dfl.loc[dfl["city"]=="Manchester", "avg_hotel_price"] = 
#dfl.loc[dfl["city"]=="Asheville", "avg_hotel_price"] = 
#dfl.loc[dfl["city"]=="Santa Cruz County", "avg_hotel_price"] = 

### Median and range of room, home etc

In [26]:
dfc = pd.DataFrame()
idx = 0
for city in dfl["city"].unique():
    df = dfl[dfl["city"]==city]
    localcurrency_usd_factor = df["localcurrency_usd_factor"].unique()[0]
    dfc.at[idx, "city"] = city
    dfc.at[idx, "median_privateroom"] = df[df["room_type"]=="Private room"]["price"].median() * localcurrency_usd_factor
    dfc.at[idx, "median_entirehome"] = df[df["room_type"]=="Entire home/apt"]["price"].median() * localcurrency_usd_factor
    dfc.at[idx, "median_sharedroom"] = df[df["room_type"]=="Shared room"]["price"].median() * localcurrency_usd_factor
    dfc.at[idx, "range_privateroom"] = (df[df["room_type"]=="Private room"]["price"].max() - df[df["room_type"]=="Private room"]["price"].min()) * localcurrency_usd_factor
    dfc.at[idx, "range_entirehome"] = (df[df["room_type"]=="Entire home/apt"]["price"].max() - df[df["room_type"]=="Entire home/apt"]["price"].min()) * localcurrency_usd_factor
    dfc.at[idx, "range_sharedroom"] = (df[df["room_type"]=="Shared room"]["price"].max() - df[df["room_type"]=="Shared room"]["price"].min()) * localcurrency_usd_factor
    dfc.at[idx, "num_listings"] = len(df)
    dfc.at[idx, "avg_hotel_price"] = df["avg_hotel_price"].unique()[0]
    dfc.at[idx, "total_income"] = df["price"].sum() * localcurrency_usd_factor
    dfc.at[idx, "frac_entireapt"] = len(df[df["room_type"]=="Entire home/apt"])/len(df)
    idx += 1

In [28]:
dfc.head(2)

,city,median_privateroom,median_entirehome,median_sharedroom,range_privateroom,range_entirehome,range_sharedroom,num_listings,avg_hotel_price,total_income,frac_entireapt
0,Barcelona,40.25,115.0,29.325,10482.25,9188.50,864.8,18531.0,172.0,2052878.8,0.465544
1,Paris,57.50,92.0,35.650,2875.00,10785.85,1419.1,59945.0,214.0,6904735.7,0.868413


### Number of listings

In [69]:
dflc = dfl.groupby("city").count()["id"].sort_values(ascending=False).reset_index()
dflc.rename(columns={"id":"numlistings"}, inplace=True)
dflc["numresidents"] = np.NaN
dflc.head(2)

,city,numlistings,numresidents
0,Paris,59945,NaN
1,London,53904,NaN


### Population

In [70]:
dflc.loc[dflc["city"]=="New York City", "numresidents"] = 8622698
dflc.loc[dflc["city"]=="London", "numresidents"] = 8787892
dflc.loc[dflc["city"]=="Paris", "numresidents"] = 10601122
dflc.loc[dflc["city"]=="Sydney", "numresidents"] = 4321535
dflc.loc[dflc["city"]=="Los Angeles", "numresidents"] = 3999759
dflc.loc[dflc["city"]=="Rome", "numresidents"] = 2879038
dflc.loc[dflc["city"]=="Berlin", "numresidents"] = 3711930
dflc.loc[dflc["city"]=="Copenhagen", "numresidents"] = 602481
dflc.loc[dflc["city"]=="Amsterdam", "numresidents"] = 856928 
dflc.loc[dflc["city"]=="Barcelona", "numresidents"] = 1602386
dflc.loc[dflc["city"]=="Madrid", "numresidents"] = 3165235
dflc.loc[dflc["city"]=="Mallorca", "numresidents"] = 859289
dflc.loc[dflc["city"]=="Melbourne", "numresidents"] = 4196198
dflc.loc[dflc["city"]=="Toronto", "numresidents"] = 2731579
dflc.loc[dflc["city"]=="Montreal", "numresidents"] = 1704694
dflc.loc[dflc["city"]=="Austin", "numresidents"] = 950715	
dflc.loc[dflc["city"]=="Vienna", "numresidents"] = 1877836
dflc.loc[dflc["city"]=="Washington, D.C.", "numresidents"] = 693972
dflc.loc[dflc["city"]=="Dublin", "numresidents"] = 1173179
dflc.loc[dflc["city"]=="Vancouver", "numresidents"] = 631486
dflc.loc[dflc["city"]=="San Diego", "numresidents"] = 1419516
dflc.loc[dflc["city"]=="Hong Kong", "numresidents"] = 7409800
dflc.loc[dflc["city"]=="Edinburgh", "numresidents"] = 1339380
dflc.loc[dflc["city"]=="Brussels", "numresidents"] = 1171828
dflc.loc[dflc["city"]=="Venice", "numresidents"] = 270884
dflc.loc[dflc["city"]=="Nashville", "numresidents"] = 667560
dflc.loc[dflc["city"]=="New Orleans", "numresidents"] = 393292
dflc.loc[dflc["city"]=="Chicago", "numresidents"] = 2716450
dflc.loc[dflc["city"]=="Athens", "numresidents"] = 664046
dflc.loc[dflc["city"]=="Boston", "numresidents"] = 685094
dflc.loc[dflc["city"]=="San Francisco", "numresidents"] = 884363
dflc.loc[dflc["city"]=="Portland", "numresidents"] = 647805
dflc.loc[dflc["city"]=="Tasmania", "numresidents"] = 519100
dflc.loc[dflc["city"]=="Denver", "numresidents"] = 704621
dflc.loc[dflc["city"]=="Seattle", "numresidents"] = 724745	
dflc.loc[dflc["city"]=="Geneva", "numresidents"] = 198979
dflc.loc[dflc["city"]=="Northern Rivers", "numresidents"] = 296531
dflc.loc[dflc["city"]=="Quebec City", "numresidents"] = 531902
dflc.loc[dflc["city"]=="Oakland", "numresidents"] = 412040
dflc.loc[dflc["city"]=="Victoria", "numresidents"] = 80017
dflc.loc[dflc["city"]=="Antwerp", "numresidents"] = 520504
dflc.loc[dflc["city"]=="Manchester", "numresidents"] = 530300
dflc.loc[dflc["city"]=="Asheville", "numresidents"] = 89121
dflc.loc[dflc["city"]=="Santa Cruz County", "numresidents"] = 262382

### Number of listings per 100 residents

In [71]:
dflc["numlistingsPerResidents"] = dflc["numlistings"]/dflc["numresidents"]*1000
dflc.head(2)

,city,numlistings,numresidents,numlistingsPerResidents
0,Paris,59945,10601122.0,5.654590
1,London,53904,8787892.0,6.133894


In [72]:
dflc = dflc.set_index("city").join(dfc.set_index("city"))[["numlistings", "numlistingsPerResidents", "median_entirehome"]].reset_index()
dflc.rename(columns={"median_entirehome":"price"}, inplace=True)
dflc["price"] = np.round(dflc["price"],0)
dflc.head(2)

,city,numlistings,numlistingsPerResidents,price
0,Paris,59945,5.654590,92.0
1,London,53904,6.133894,145.0


In [73]:
listings1_excluded_cities = ["Austin", "Antwerp", "Oakland", "Portland", "Athens", "Montreal", 
                             "San Diego", "Nashville", "Athens", "Quebec city", "Manchester", "Asheville"]
dflc = dflc[-dflc["city"].isin(listings1_excluded_cities)]
dflc.head(2)

,city,numlistings,numlistingsPerResidents,price
0,Paris,59945,5.654590,92.0
1,London,53904,6.133894,145.0


In [51]:
df = dflc.sort_values("numlistings", ascending=False)[:30]
with open('../code/data/num_listings_price_circle_packing.csv', 'w') as fp:  
    writer = csv.writer(fp, quoting=csv.QUOTE_NONE, delimiter="\t")
    writer.writerow(["id","value","city","price"])
    for i in range(len(df)):
        line = 'city.' + df.iloc[i]["city"] + ',' + str(df.iloc[i]["numlistings"]) + \
               "," + df.iloc[i]["city"] + "," + str(df.iloc[i]["price"])
        writer.writerow([line])        

In [75]:
df = dflc.sort_values("numlistingsPerResidents", ascending=False)[:30]
df["numlistingsPerResidents"] = np.round(df["numlistingsPerResidents"],2)
with open('../code/data/num_listings_per_residents_price_circle_packing.csv', 'w') as fp:  
    writer = csv.writer(fp, quoting=csv.QUOTE_NONE, delimiter="\t")
    writer.writerow(["id","value","city","price"])
    for i in range(len(df)):
        line = 'city.' + df.iloc[i]["city"] + ',' + str(df.iloc[i]["numlistingsPerResidents"]) + \
               "," + df.iloc[i]["city"] + "," + str(df.iloc[i]["price"])
        writer.writerow([line])        

In [78]:
df

,city,numlistings,numlistingsPerResidents,price
7,Copenhagen,20545,34.10,111.0
24,Venice,6027,22.25,138.0
8,Amsterdam,18547,21.64,150.0
39,Victoria,1691,21.13,89.0
11,Mallorca,14858,17.29,131.0
35,Geneva,3060,15.38,110.0
26,New Orleans,5215,13.26,150.0
9,Barcelona,18531,11.56,115.0
18,"Washington, D.C.",7788,11.22,160.0
20,Vancouver,6651,10.53,108.0
